In [22]:
import re
import requests
import codecs
import time
import random
from bs4 import BeautifulSoup

'''
absolute='https://movie.douban.com/subject/26322642/comments'
absolute_url='https://movie.douban.com/subject/26322642/comments?start=23&limit=20&sort=new_score&status=P&precent_type='
url='https://movie.douban.com/subject/26322642/comments?start={}&limit=20&sort=new_score&status=P'

headers = {
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3135.4 Safari/537.36"
}

'''
from selenium import webdriver
browser = webdriver.Chrome(executable_path='C:/Users/pujing/AppData/Local/Google/Chrome/Application/chromedriver.exe')
browser.get('https://movie.douban.com/subject/26322642/comments?start=0&limit=20&sort=new_score&status=P&percent_type=')
time.sleep(3)

In [10]:
html=browser.page_source

In [11]:
soup=BeautifulSoup(html,'lxml')

In [12]:
comment_list=soup.select('.mod-bd')

In [13]:
next_page=soup.select('#paginator .next')[0].get('href')

In [14]:
next_page

'?start=20&limit=20&sort=new_score&status=P&percent_type='

In [15]:
browser.get('https://movie.douban.com/subject/26322642/comments'+next_page)
html=browser.page_source
soup=BeautifulSoup(html,'lxml')
comment_list=soup.select('.mod-bd')
next_page=soup.select('#paginator .next')[0].get('href')

In [16]:
next_page

'?start=40&limit=20&sort=new_score&status=P&percent_type='

In [23]:
def jiexi(soup,res_users,res_scores,res_times,res_comment):
    comment_info=soup.select('.mod-bd .comment-info > a')
    for i in range(len(comment_info)):
        res_users.append(comment_info[i].get_text())

    comment_pingfen=soup.select('.mod-bd .comment-info > span')
    fen_ls = '<span class="allstar.*?" title="(.*?)"></span>'
    fen_pattern = re.compile(fen_ls, re.S)
    fen_lists = re.findall(fen_pattern, str(comment_pingfen))
    for li in fen_lists:
        #print(li)
        res_scores.append(li)

    #获取一个页面的评论时间
    time_ls = '<span class="comment-time " title="(.*?)">.*?</span>'
    time_pattern = re.compile(time_ls, re.S)
    time_lists = re.findall(time_pattern, str(comment_pingfen))
    for li in time_lists:
        #print(li)
        res_times.append(li)

    comment_lists=soup.select('.mod-bd p')
    for i in(range(len(comment_lists))):
        res_comment.append(comment_lists[i].get_text())

In [24]:
def get_next_html(browser):
    html=browser.page_source
    soup=BeautifulSoup(html,'lxml')
    comment_list=soup.select('.mod-bd')
    next_page=soup.select('#paginator .next')[0].get('href')
    return next_page,soup

In [25]:
import pandas as pd
import time

In [36]:
#next_page='?start=0&limit=20&sort=new_score&status=P&percent_type='
def main():
    res_users=[]
    res_scores=[]
    res_times=[]
    res_comment=[]
    i=0;
    item=0
    #while(next_page!=[]):
    browser.get('https://movie.douban.com/subject/26322642/comments?start=0&limit=20&sort=new_score&status=P&percent_type=')
    for j in range(100):
        i=i+1
        next_page,soup=get_next_html(browser)
        if(next_page==None):
            break
        browser.get('https://movie.douban.com/subject/26322642/comments'+next_page)
        jiexi(soup,res_users,res_scores,res_times,res_comment)
        result=pd.DataFrame([res_users,res_scores,res_times,res_comment])
        result=result.T
        result.to_excel('F:\\hua\\mo\\aa\\result.xlsx')
        time.sleep(1)
    

In [37]:
main()